## Convert Keras checkpoint to Tensorflow savedmodel and Graphdef

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf


from tensorflow import keras
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np

print('tf version', tf.__version__)

tf version 2.6.0


## Convert a Keras h5 model to Tensorflow savedmodel

In [ ]:
keras_model_ckpt_path = "/root/model/clf"

In [2]:
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
MAX_SEQ_LEN = 256
model = TFAutoModelForSequenceClassification.from_pretrained(keras_model_ckpt_path)
callable = tf.function(model.call)
concrete_function = callable.get_concrete_function([tf.TensorSpec([None, MAX_SEQ_LEN], tf.int32, name="input_ids"), tf.TensorSpec([None, MAX_SEQ_LEN], tf.int32, name="attention_mask")])
model.save('/root/saved_model/distilbert/1', signatures=concrete_function)


All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at /root/model/clf.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


INFO:tensorflow:Assets written to: /root/saved_model/distilbert/1/assets


INFO:tensorflow:Assets written to: /root/saved_model/distilbert/1/assets


## Convert a keras h5 model to Graphdef 

In [4]:

## convert transformers keras model 
MAX_SEQ_LEN = 256
model = TFAutoModelForSequenceClassification.from_pretrained(keras_model_ckpt_path)
callable = tf.function(model.call)
full_model = callable.get_concrete_function([tf.TensorSpec([None, MAX_SEQ_LEN], tf.int32, name="input_ids"), tf.TensorSpec([None, MAX_SEQ_LEN], tf.int32, name="attention_mask")])

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

# inspect the layers operations inside your frozen graph definition and see the name of its input and output tensors
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 50)
print("Frozen model layers: ")
for layer in layers:
    print(layer)

print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

# Save frozen graph from frozen ConcreteFunction to hard drive
# serialize the frozen graph and its text representation to disk.
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="/root/frozen_models",
                  name="simple_frozen_graph.pb",
                  as_text=False)

#Optional
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="/root/frozen_models",
                  name="simple_frozen_graph.pbtxt",
                as_text=True)

model.summary()


Some layers from the model checkpoint at /root/model/clf were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /root/model/clf and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--------------------------------------------------
Frozen model layers: 
input_ids
attention_mask
distilbert/transformer/layer_._5/ffn/Gelu/mul/x
distilbert/transformer/layer_._4/ffn/Gelu/mul/x
distilbert/transformer/layer_._3/ffn/Gelu/mul/x
distilbert/transformer/layer_._2/ffn/Gelu/mul/x
distilbert/transformer/layer_._1/ffn/Gelu/mul/x
distilbert/transformer/layer_._0/ffn/Gelu/mul/x
distilbert/embeddings/Gather_1/resource
distilbert/embeddings/range/start
distilbert/embeddings/range/limit
distilbert/embeddings/range/delta
distilbert/embeddings/range
distilbert/embeddings/ExpandDims/dim
distilbert/embeddings/ExpandDims
distilbert/embeddings/Gather_1/axis
distilbert/embeddings/Gather_1
distilbert/embeddings/Identity_1
distilbert/embeddings/Gather/resource
distilbert/embeddings/Gather/axis
distilbert/embeddings/Gather
distilbert/embeddings/Identity
distilbert/embeddings/Shape
distilbert/embeddings/strided_slice/stack
distilbert/embeddings/strided_slice/stack_1
distilbert/embeddings/stride